# 2.8 Control de Versión de Datos

**Resumen Ejecutivo**: Cómo rastrear cambios en datos de referencia (dimensiones) y hechos críticos mediante columnas de auditoría, tablas de historial y hashes de fila para detectar modificaciones. El objetivo es asegurar trazabilidad, reproducibilidad y calidad en pipelines analíticos y operativos.

---

## 🎯 Objetivos de Aprendizaje

1. Entender patrones básicos de control de versiones en datos (timestamps, soft delete, tablas de historial)
2. Implementar columnas de auditoría (`created_at`, `updated_at`) y estrategias de hash de fila
3. Detectar cambios en dimensiones lentas y preparar datos para auditorías
4. Evitar errores comunes (zonas horarias, hashes inseguros, crecimiento sin control)

---

## 🧰 Prerrequisitos

- Conocer DDL básico (`ALTER TABLE`, tipos de datos datetime)
- Familiaridad con tablas dimensionales (`dim_clientes`, `dim_productos`)
- Comprender conceptos de integridad referencial y ETL/ELT

---

## 🧭 Patrón 1: Columnas de Auditoría

Añadir columnas `created_at` y `updated_at` permite rastrear cuándo se creó o modificó un registro. Útil para conciliaciones, depuración y SLA de datos.

**Buenas prácticas**:
- Usar `DATETIME2` o `DATETIMEOFFSET` para preservar precisión y zona horaria.
- Actualizar `updated_at` desde aplicación o trigger controlado (evitar overhead excesivo).
- Documentar la semántica (qué eventos actualizan la columna).

➡️ En el siguiente bloque creamos una columna `updated_at` como ejemplo.

---

## 🧭 Patrón 2: Hash de Fila para Detección de Cambios

Los hashes permiten detectar cambios sin comparar campo a campo. Se calculan sobre columnas relevantes y se comparan con el hash previo.

**Buenas prácticas**:
- Evitar MD5/SHA1 para usos legales; preferir SHA256 si se requiere seguridad.
- Incluir solo columnas de negocio relevantes (excluir audit fields para evitar falsos positivos).
- Guardar el hash previo para detectar modificaciones entre cargas.

➡️ En el siguiente bloque calculamos un hash de fila con `CHECKSUM` (solo demostrativo).

---

## 🧪 Ejemplo 1: Añadir columna `updated_at`

Descripción: agregamos columna de timestamp para registrar la última actualización. Útil para ETL incremental y auditoría básica.


In [ ]:
-- Ejemplo conceptual: añadir columna updated_at
-- ALTER TABLE dbo.dim_clientes ADD updated_at DATETIME2 NULL;

---

## 🧪 Ejemplo 2: Hash de Fila (detección de cambios)

Descripción: calculamos un hash ligero con `CHECKSUM` para detectar si cambió algún campo relevante del cliente. En producción, almacenar el hash previo y comparar para identificar modificaciones.

**Resultado esperado**: columna `hash_fila` que cambia cuando se altera `nombre`, `segmento` o `fecha_alta`.

**Nota**: `CHECKSUM` es demostrativo; para mayor robustez usar `HASHBYTES('SHA2_256', ...)`.

In [ ]:
-- Hash para detectar cambios (MD5/Checksum) - demostración conceptual
SELECT cliente_id, nombre, segmento,
       CHECKSUM(nombre, segmento, fecha_alta) AS hash_fila
FROM dbo.dim_clientes;

---

## 🧪 Ejercicios y Retos

- 🟢 Detectar cambios de clientes: calcular hash actual, comparar con hash previo (simulado) y listar posibles modificaciones.
- 🟠 Diseñar tabla historial: `cliente_id, campo, valor_anterior, valor_nuevo, fecha_cambio, usuario` con índices por `cliente_id` y `fecha_cambio`.
- 🔴 Simular control de versiones de precios: usar CTE con versión anterior y actual para reportar diferencias y fechas de vigencia.

---

## 🛠️ Aplicación Práctica

- **Dimensiones con cambios lentos (SCD-lite)**: Usa `updated_at` + hash para detectar cambios y generar versiones solo cuando el hash difiera.
- **Auditoría de catálogos**: Registrar en tabla historial quién cambió precios o segmentos y cuándo.
- **ETL incremental**: Filtrar registros con `updated_at > @last_run` para reducir ventanas de carga.

---

## 💼 Perspectiva de Negocio

- **Trazabilidad y cumplimiento**: Facilita auditorías (SOX, GDPR) al evidenciar quién cambió qué y cuándo.
- **Menos incidentes**: Detectar cambios inesperados en catálogos evita errores de facturación o reportes desviados.
- **Eficiencia operativa**: ETL incremental reduce tiempos de carga y costo computacional.

---

## ✅ Conclusiones

- El control de versiones en datos se apoya en metadatos (timestamps) y huellas (hashes) para rastrear cambios y explicarlos.
- Combinar auditoría (quién/cuándo) con comparación de hash (qué cambió) permite diferenciar actualizaciones esperadas de anomalías.
- Mantener índices adecuados y pruebas periódicas evita que el control de cambios degrade el rendimiento de ETL/ELT.

---

## ⚠️ Errores Comunes

- No registrar zona horaria (`DATETIMEOFFSET`); dificulta conciliaciones.
- Usar hashes inseguros (MD5/SHA1) para fines de auditoría legal.
- Ignorar impacto del historial en tamaño: archivar o particionar tablas antiguas.

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel 2  
**Módulo**: 2.8 Control de Versión de Datos  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 **Nota sobre IA**: Este material fue estructurado con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude); el contenido fue validado y curado por especialistas para evitar alucinaciones y asegurar aplicabilidad práctica.

---
## Navegación
[⬅️ Anterior](07_optimizar_consultas_basico.ipynb) | [Siguiente ➡️](09_integracion_fuentes.ipynb)
---
